# Huggingface Sagemaker-sdk - Distributed Training Demo
### Distributed Summarization with `transformers` scripts + `Trainer` and `samsum` dataset

1. [Tutorial](#Tutorial)  
2. [Set up a development environment and install sagemaker](#Set-up-a-development-environment-and-install-sagemaker)
    1. [Installation](#Installation)  
    2. [Development environment](#Development-environment)  
    3. [Permissions](#Permissions) 
4. [Choose 🤗 Transformers `examples/` script](#Choose-%F0%9F%A4%97-Transformers-examples/-script)  
1. [Configure distributed training and hyperparameters](#Configure-distributed-training-and-hyperparameters)  
2. [Create a `HuggingFace` estimator and start training](#Create-a-HuggingFace-estimator-and-start-training)   
3. [Upload the fine-tuned model to huggingface.co](#Upload-the-fine-tuned-model-to-huggingface.co)

# Tutorial

We will use the new [Hugging Face DLCs](https://github.com/aws/deep-learning-containers/tree/master/huggingface) and [Amazon SageMaker extension](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#huggingface-estimator) to train a distributed Seq2Seq-transformer model on `summarization` using the `transformers` and `datasets` libraries and upload it afterwards to [huggingface.co](http://huggingface.co) and test it.

As [distributed training strategy](https://huggingface.co/transformers/sagemaker.html#distributed-training-data-parallel) we are going to use [SageMaker Data Parallelism](https://aws.amazon.com/blogs/aws/managed-data-parallelism-in-amazon-sagemaker-simplifies-training-on-large-datasets/), which has been built into the [Trainer](https://huggingface.co/transformers/main_classes/trainer.html) API. To use data-parallelism we only have to define the `distribution` parameter in our `HuggingFace` estimator.

```python
# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}
```

In this tutorial, we will use an Amazon SageMaker Notebook Instance for running our training job. You can learn [here how to set up a Notebook Instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html).

**What are we going to do:**

- Set up a development environment and install sagemaker
- Chose 🤗 Transformers `examples/` script
- Configure distributed training and hyperparameters
- Create a `HuggingFace` estimator and start training
- Upload the fine-tuned model to [huggingface.co](http://huggingface.co)
- Test inference

### Model and Dataset

We are going to fine-tune [facebook/bart-base](https://huggingface.co/facebook/bart-base) on the [samsum](https://huggingface.co/datasets/samsum) dataset. *"BART is sequence-to-sequence model trained with denoising as pretraining objective."* [[REF](https://github.com/pytorch/fairseq/blob/master/examples/bart/README.md)]

The `samsum` dataset contains about 16k messenger-like conversations with summaries. 

```python
{'id': '13818513',
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'dialogue': "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"}
```

_**NOTE: You can run this demo in Sagemaker Studio, your local machine or Sagemaker Notebook Instances**_

# Set up a development environment and install sagemaker

## Installation

_**Note:** The use of Jupyter is optional: We could also launch SageMaker Training jobs from anywhere we have an SDK installed, connectivity to the cloud and appropriate permissions, such as a Laptop, another IDE or a task scheduler like Airflow or AWS Step Functions._

In [ ]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade
#!apt install git-lfs

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs -y
!git lfs install

## Development environment 

**upgrade ipywidgets for `datasets` library and restart kernel, only needed when prerpocessing is done in the notebook**

In [ ]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True) # has to restart kernel so changes are used

In [1]:
import sagemaker.huggingface

## Permissions

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [2]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::859830842924:role/service-role/AmazonSageMaker-ExecutionRole-20211013T235227
S3 bucket used for storing artifacts: sagemaker-us-east-1-859830842924


## Choose 🤗 Transformers `examples/` script

The [🤗 Transformers repository](https://github.com/huggingface/transformers/tree/master/examples) contains several `examples/`scripts for fine-tuning models on tasks from `language-modeling` to `token-classification`. In our case, we are using the `run_summarization.py` from the `seq2seq/` examples. 

_**Note**: you can use this tutorial identical to train your model on a different examples script._

Since the `HuggingFace` Estimator has git support built-in, we can specify a [training script that is stored in a GitHub repository](https://sagemaker.readthedocs.io/en/stable/overview.html#use-scripts-stored-in-a-git-repository) as `entry_point` and `source_dir`.

We are going to use the `transformers 4.4.2` DLC which means we need to configure the `v4.4.2` as the branch to pull the compatible example scripts.

In [3]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'} # v4.6.1 is referring to the `transformers_version` you use in the estimator.

## Configure distributed training and hyperparameters

Next, we will define our `hyperparameters` and configure our distributed training strategy. As hyperparameter, we can define any [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#seq2seqtrainingarguments) and the ones defined in [run_summarization.py](https://github.com/huggingface/transformers/tree/master/examples/seq2seq#sequence-to-sequence-training-and-evaluation). 

In [10]:
# hyperparameters, which are passed into the training job
hyperparameters={'per_device_train_batch_size': 1,
                 'per_device_eval_batch_size': 1,
                 'model_name_or_path': 'facebook/bart-large-cnn',
                 'dataset_name': 'big_patent', #dropping to test S3 based dataset.
                 'dataset_config_name': 'g',
                 'do_train': True,
                 'do_eval': True,
                 'do_predict': True,
                 'predict_with_generate': True,
                 'output_dir': '/opt/ml/model',
                 'num_train_epochs': 3,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': True,
                 'cache_dir': 'opt/ml/input',
                 'text_column': 'description',
                 'summary_column': 'abstract',
                }

# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

## Create a `HuggingFace` estimator and start training

In [14]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_summarization.py', # script
      source_dir='./examples/pytorch/summarization', # relative path to example
      git_config=git_config,
      instance_type='ml.p3.16xlarge',
      instance_count=1,
      transformers_version='4.12',
      pytorch_version='1.9',
      py_version='py38',
      role=role,
      hyperparameters = hyperparameters,
      distribution = distribution,
      volume_size=200,
      cache_dir='opt/ml/input'
)

In [15]:
# starting the train job
huggingface_estimator.fit()
#huggingface_estimator.fit({'train': 's3://robcost-bigpatent/train.tar.gz','test': 's3://robcost-bigpatent/test.tar.gz'})

2022-01-17 12:28:53 Starting - Starting the training job...
2022-01-17 12:29:16 Starting - Launching requested ML instancesProfilerReport-1642422525: InProgress
.........
2022-01-17 12:30:43 Starting - Preparing the instances for training.........
2022-01-17 12:32:17 Downloading - Downloading input data...
2022-01-17 12:32:37 Training - Downloading the training image..............................
2022-01-17 12:37:48 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-17 12:37:48,981 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-17 12:37:49,057 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-01-17 12:37:49,063 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2022-01-17 12:37:49,063 sagemaker_pytorch_container.training INFO     Invoking 

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2022-01-17-12-28-45-434: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "mpirun --host algo-1 -np 8 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -mca plm_rsh_num_concurrent 1 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_SINGLENODE=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/opt/conda/lib/python3.8/site-packages/gethostname.cpython-38-x86_64-linux-gnu.so smddprun /opt/conda/bin/python3.8 -m mpi4py run_summarization.py --cache_dir opt/ml/input --dataset_config_name g --dataset_name big_patent --do_eval True --do_predict True --do_train True --fp16 True --learning_rate 5e-05 --model_name_or_path facebook/bart-

## Deploying the endpoint

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

Then, we use the returned predictor object to call the endpoint.

In [ ]:
patent = '''BACKGROUND OF THE DISCLOSURE

      1. Technical Field of the Disclosure
      This embodiment relates in general to electric cars. More specifically, the preferred embodiment relates to an electric car having a vertical row of seats.
      2. Description of the Related Art
      An electric car is powered by an electric motor using electrical energy stored in batteries or other charged devices. Electric cars are environment friendly as it do not produce any harmful gases such as carbon monoxide, organic compounds, hydro carbons etc. Electric cars are economical because of very low maintenance and operating costs.
      Conventional electric cars have considerable drawbacks. For example, an existing electric car includes an electric drive and at least one connected electrical energy storage module. A guide extends longitudinally along the motor vehicle, and supports the storage module in a manner relative to the motor vehicle. Conventional electric cars employ horizontal rows of seats consuming excess space.
      Another existing electric vehicle is capable of carrying at least two passengers and has at least three wheels. Passengers sit in tandem and most of the batteries or fuel cell systems are located to the sides of the passengers. This vehicle has an aerodynamically shaped body with substantially reduced frontal area and drag. The body is lightweight, made from shock absorbing materials and structures, and has pressure-airless tires, which enhances the safety of the passengers. The vehicle also includes an advanced hydrogen-electric hybrid propulsion system with quick refueling from existing infrastructure and various additional optional features and systems. However the batteries or fuel cell systems are placed to the sides of the passengers which increases the overall the dimension and weight of the vehicle.
      Yet another existing electrical car embodiment is comprised of bodywork with ground-engaging wheels for vehicle motion over the ground, the bodywork contains an electric motor to drive the vehicle via the wheels and batteries to power the electric motor.
      This embodiment provides an additional energy generation means comprised of a tunnel extending through the bodywork. It includes a turbine fan/alternator set located in the tunnel at the rear of the vehicle where electrical energy is generated during vehicle motion to charge the batteries. This results in improved performance of the vehicle, especially with regard to its range. The inlet to the tunnel at the vehicle front constitutes the major portion of the vehicles frontal area. A special alternator is provided, and the vehicle can also include a solar cell means for battery charging. However, the seats are placed far apart which consumes a lot of space.
      Hence, it can be seen, that there is a need for an electric car that contains a vertical rows of seats. This needed electric car would save more space than existing models to form a vehicle of smaller size with more passengers. Moreover, this needed electric car would consume less power and use a less bulky charging means during transportation. The present embodiment accomplishes these objectives.
SUMMARY OF THE DISCLOSURE

      To minimize the limitations found in the prior art, and to minimize other limitations that will be apparent upon the reading of the specifications, the present invention provides an electric car having a vertical row of seat for accommodating at least one passenger. The electric car comprises a center console placed at an interior forepart of the electric car. A steering is attached to the center console. A rectangular seat is mounted on a rectangular box, the rectangular box being longitudinally placed at a center part of the electric car. A storage area is enclosed within the rectangular box, the storage area includes a personal storage and a battery storage. The battery storage includes a battery pack having a set of rechargeable batteries. The battery pack is used to power the electric car. A plug point is located at a rear end of the electric car for charging the battery pack. A pair of rotatable front wheels and a pair of rotatable back wheels are provided for ensuring smooth movement of the electric car. Both the center console and the steering of the electric car are aligned with the rectangular seat. The rectangular seat is mounted on the rectangular box having the storage area in order to save space. The electric car is configured to have a compact seating arrangement. A back support may be provided for a driver's rectangular seat. The electric car is specially designed to achieve better performance by increasing the energy efficiency and reducing the power consumption. The at least one passenger can be seated facing front direction with one leg on each side of the rectangular seat. The electric car is capable of accommodating more passengers with relatively small size and with minimum power consumption during transportation.
      In alternate embodiment of the present invention at least one step and ladder seat is positioned at the center part of the electric car. The electric car comprises a car body having a front end, a rear end, a top portion and a bottom portion. A center console placed at an interior forepart of the electric car. A steering is attached to the center console. At least one step and ladder seat is positioned at the center part of the electric car for accommodating at least one passenger. In addition to the at least one step and ladder seat, a horizontal row of seats for accommodating a pair of passengers is located at the rear end of the vehicle. A battery storage having a battery pack is positioned at the bottom portion of step and ladder seat for powering the electric car. A plug point is located at a rear end of the electric car for charging the battery pack. A personal storage is placed at the rear end of horizontal row of seat.
      One objective of the invention is to provide an electric car capable of accommodating four or more passengers with relatively smaller vehicle size.
      Another objective of the invention is to provide an electric car with compact seating arrangement.
      A third objective of the invention is to provide an electric car with better performance by increasing the energy efficiency and reducing the power consumption.
      These and other advantages and features of the present invention are described with specificity so as to make the present invention understandable to one of ordinary skill in the art.'''

data= {"inputs":patent}

predictor.predict(data)

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_endpoint()

## Upload the fine-tuned model to [huggingface.co](http://huggingface.co)

We can download our model from Amazon S3 and unzip it using the following snippet.



In [ ]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = 'my_bart_model'

os.makedirs(local_path, exist_ok = True)

# download model from S3
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path=local_path, # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

# unzip model
tar = tarfile.open(f"{local_path}/model.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()
os.remove(f"{local_path}/model.tar.gz")

Before we are going to upload our model to [huggingface.co](http://huggingface.co) we need to create a `model_card`. The `model_card` describes the model includes hyperparameters, results and which dataset was used for training. To create a `model_card` we create a `README.md` in our `local_path`

In [ ]:
# read eval and test results 
with open(f"{local_path}/eval_results.json") as f:
    eval_results_raw = json.load(f)
    eval_results={}
    eval_results["eval_rouge1"] = eval_results_raw["eval_rouge1"]
    eval_results["eval_rouge2"] = eval_results_raw["eval_rouge2"]
    eval_results["eval_rougeL"] = eval_results_raw["eval_rougeL"]
    eval_results["eval_rougeLsum"] = eval_results_raw["eval_rougeLsum"]

with open(f"{local_path}/test_results.json") as f:
    test_results_raw = json.load(f)
    test_results={}
    test_results["test_rouge1"] = test_results_raw["test_rouge1"]
    test_results["test_rouge2"] = test_results_raw["test_rouge2"]
    test_results["test_rougeL"] = test_results_raw["test_rougeL"]
    test_results["test_rougeLsum"] = test_results_raw["test_rougeLsum"]

In [ ]:
After we extract all the metrics we want to include we are going to create our `README.md`. Additionally to the automated generation of the results table we add the metrics manually to the `metadata` of our model card under `model-index`

In [ ]:
print(eval_results)
print(test_results)

In [ ]:
import json


MODEL_CARD_TEMPLATE = """
---
language: en
tags:
- sagemaker
- bart
- summarization
license: apache-2.0
datasets:
- samsum
model-index:
- name: {model_name}
  results:
  - task: 
      name: Abstractive Text Summarization
      type: abstractive-text-summarization
    dataset:
      name: "SAMSum Corpus: A Human-annotated Dialogue Dataset for Abstractive Summarization" 
      type: samsum
    metrics:
       - name: Validation ROGUE-1
         type: rogue-1
         value: 42.621
       - name: Validation ROGUE-2
         type: rogue-2
         value: 21.9825
       - name: Validation ROGUE-L
         type: rogue-l
         value: 33.034
       - name: Test ROGUE-1
         type: rogue-1
         value: 41.3174
       - name: Test ROGUE-2
         type: rogue-2
         value: 20.8716
       - name: Test ROGUE-L
         type: rogue-l
         value: 32.1337
widget:
- text: | 
    Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
    Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
    Jeff: ok.
    Jeff: and how can I get started? 
    Jeff: where can I find documentation? 
    Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face 
---
## `{model_name}`
This model was trained using Amazon SageMaker and the new Hugging Face Deep Learning container.
For more information look at:
- [🤗 Transformers Documentation: Amazon SageMaker](https://huggingface.co/transformers/sagemaker.html)
- [Example Notebooks](https://github.com/huggingface/notebooks/tree/master/sagemaker)
- [Amazon SageMaker documentation for Hugging Face](https://docs.aws.amazon.com/sagemaker/latest/dg/hugging-face.html)
- [Python SDK SageMaker documentation for Hugging Face](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html)
- [Deep Learning Container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-training-containers)
## Hyperparameters
    {hyperparameters}
## Usage
    from transformers import pipeline
    summarizer = pipeline("summarization", model="philschmid/{model_name}")
    conversation = '''Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
    Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
    Jeff: ok.
    Jeff: and how can I get started? 
    Jeff: where can I find documentation? 
    Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face                                           
    '''
    nlp(conversation)
## Results
| key | value |
| --- | ----- |
{eval_table}
{test_table}
"""

# Generate model card (todo: add more data from Trainer)
model_card = MODEL_CARD_TEMPLATE.format(
    model_name=f"{hyperparameters['model_name_or_path'].split('/')[1]}-{hyperparameters['dataset_name']}",
    hyperparameters=json.dumps(hyperparameters, indent=4, sort_keys=True),
    eval_table="\n".join(f"| {k} | {v} |" for k, v in eval_results.items()),
    test_table="\n".join(f"| {k} | {v} |" for k, v in test_results.items()),
)
with open(f"{local_path}/README.md", "w") as f:
    f.write(model_card)

After we have our unzipped model and model card located in `my_bart_model` we can use the either `huggingface_hub` SDK to create a repository and upload it to [huggingface.co](http://huggingface.co) or go to https://huggingface.co/new an create a new repository and upload it.

In [ ]:
from getpass import getpass
from huggingface_hub import HfApi, Repository

hf_username = "philschmid" # your username on huggingface.co
hf_email = "philipp@huggingface.co" # email used for commit
repository_name = f"{hyperparameters['model_name_or_path'].split('/')[1]}-{hyperparameters['dataset_name']}" # repository name on huggingface.co
password = getpass("Enter your password:") # creates a prompt for entering password

# get hf token
token = HfApi().login(username=hf_username, password=password)

# create repository
repo_url = HfApi().create_repo(token=token, name=repository_name, exist_ok=True)

# create a Repository instance
model_repo = Repository(use_auth_token=token,
                        clone_from=repo_url,
                        local_dir=local_path,
                        git_user=hf_username,
                        git_email=hf_email)

# push model to the hub
model_repo.push_to_hub()

print(f"https://huggingface.co/{hf_username}/{repository_name}")